# Calculate CTE Covariance

In [1]:
%matplotlib inline

# system imports
from matplotlib import pylab as plt
import numpy
import os

# LSST stack imports
from lsst.daf.persistence import Butler
import lsst.afw.display as afwDisplay
from lsst.ip.isr import IsrTask
import lsst.afw.geom as afwGeom
import lsst.afw.math as afwMath


# Firefly client imports
from IPython.display import IFrame

In [2]:
import matplotlib
matplotlib.rcParams['figure.dpi'] = 120

### Set up the config for the ISR task.  This essentially turns off all processing other than overscan and bias correction.

In [3]:
isr_config = IsrTask.ConfigClass()

isr_config.doDark=False
isr_config.doFlat=False
isr_config.doFringe=False
isr_config.doDefect=False
isr_config.doAddDistortionModel=False
isr_config.doLinearize=False
isr_config.doSaturationInterpolation=False

### Construct the `IsrTask` with the above configuration

In [4]:
isr = IsrTask(config=isr_config)

In [5]:
BOOTCAMP_REPO_DIR = '/project/bootcamp/repo_RTM-007/'
butler = Butler(BOOTCAMP_REPO_DIR)
visits = butler.queryMetadata('raw', ['visit'], dataId={'imageType': 'FLAT', 'testType': 'SFLAT_500'})

In [6]:
# Handle non-even series of images
# Tried to do something more clever, but not fully worked through yet
#etimes = []
#for visit in visits:
#    dId = {'visit': visit, 'detector': 2}
#    raw = butler.get('raw', **dId)
#    etimes.append(raw.getInfo().getVisitInfo().getExposureTime())
#exposure_times = numpy.array(etimes)
#unique_times, unique_indicies = numpy.unique(exposure_times, return_index=True)
#print(unique_indicies)
#indicies = unique_indicies.tolist() + [exposure_times.size - 1]

# Brute force for RTM-007, but inaccurate for all cases
bad_visits = []
for visit1, visit2 in zip(visits[:-1:2], visits[1::2]):
    dId = {'visit': visit1, 'detector': 2}
    raw1 = butler.get('raw', **dId)
    time1 = raw1.getInfo().getVisitInfo().getExposureTime()
    
    # Get ISR data for second image
    dId = {'visit': visit2, 'detector': 2}
    raw2 = butler.get('raw', **dId)
    time2 = raw2.getInfo().getVisitInfo().getExposureTime()
    if abs(time1 - time2) > 0.01:
        print("Mismatched exptimes")
        bad_visits.append(visit1)

print(len(visits))
for bad_visit in bad_visits:
    visits.remove(bad_visit)
print(len(visits))

Mismatched exptimes
35
34


In [8]:
gain = {}
exp_time = {}

visits = visits[0:2]

i = 1
for visit1, visit2 in zip(visits[:-1:2], visits[1::2]): # loop over pairs of images
    # Get ISR data for first image
    dId = {'visit': visit1, 'detector': 2}
    raw1 = butler.get('raw', **dId)
    bias1 = butler.get('bias', **dId)
    time1 = raw1.getInfo().getVisitInfo().getExposureTime()
    
    # Get ISR data for second image
    dId = {'visit': visit2, 'detector': 2}
    raw2 = butler.get('raw', **dId)
    bias2 = butler.get('bias', **dId)
    time2 = raw2.getInfo().getVisitInfo().getExposureTime()
    if abs(time1 - time2) > 0.01:
        print("Mismatched exptimes")
        continue
    
    # run ISR on both images
    result1 = isr.run(raw1, bias=bias1)
    result2 = isr.run(raw2, bias=bias2)
    
    detector = result1.exposure.getDetector()
    amp = detector[3]

    sub_im1 = result1.exposure.getMaskedImage()[amp.getBBox()]
    #arr1 = sub_im1.getImage().getArray()
    sub_im2 = result2.exposure.getMaskedImage()[amp.getBBox()]
    #arr2 = sub_im2.getImage().getArray()
    #print(sub_im1.getImage().getArray().shape)
    
    stats_im1 = afwMath.makeStatistics(sub_im1, afwMath.MEDIAN)
    stats_im2 = afwMath.makeStatistics(sub_im2, afwMath.MEDIAN)
    
    avg_flux1 = stats_im1.getValue(afwMath.MEDIAN)
    avg_flux2 = stats_im2.getValue(afwMath.MEDIAN)
    print(avg_flux1, avg_flux2)
    
    avg_flux = (avg_flux1 + avg_flux2) / 2
    
    sub_im1 /= avg_flux1
    sub_im2 /= avg_flux2
    
    # From RHL, 1/g = <(I1-I2)**2/(I1+I2)>
    diff_im = sub_im1.clone()
    diff_im -= sub_im2
    
    diff_im /= avg_flux
    
    sum_im = sub_im1.clone()
    sum_im += sub_im2
    
    diff_im *= diff_im
    diff_im /= sum_im

(minimum=(1536, 0), maximum=(2047, 2001))
1050.132356432354 1061.287303008515
